In [13]:
!pip install pandas scikit-learn
import pandas as pd

# Load the BankChurners dataset again
bank_data = pd.read_csv("/Users/neeharbujimalla/Downloads/BankChurners.csv")

# Display the first few rows to understand its structure
bank_data.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Extract column names as documents
documents = bank_data.columns

# Preprocess and vectorize the column names
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

# Apply LDA
num_topics = 5  # Adjust based on how many topics you believe there might be
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Extract and display the top words for each topic
topic_words = {}
for topic, comp in enumerate(lda.components_):
    word_idx = comp.argsort()[-5:][::-1]
    topic_words[topic] = [vectorizer.get_feature_names_out()[i] for i in word_idx]

topic_words

{0: ['months_on_book',
  'marital_status',
  'contacts_count_12_mon',
  'naive_bayes_classifier_attrition_flag_card_category_contacts_count_12_mon_dependent_count_education_level_months_inactive_12_mon_1',
  'total_revolving_bal'],
 1: ['total_amt_chng_q4_q1',
  'total_trans_ct',
  'income_category',
  'total_ct_chng_q4_q1',
  'avg_open_to_buy'],
 2: ['months_inactive_12_mon',
  'education_level',
  'credit_limit',
  'card_category',
  'clientnum'],
 3: ['total_relationship_count',
  'attrition_flag',
  'total_trans_amt',
  'clientnum',
  'total_revolving_bal'],
 4: ['dependent_count',
  'customer_age',
  'avg_utilization_ratio',
  'naive_bayes_classifier_attrition_flag_card_category_contacts_count_12_mon_dependent_count_education_level_months_inactive_12_mon_2',
  'clientnum']}

In [15]:
# Extract and display the top words for each topic from the column names using a workaround
feature_names = list(vectorizer.vocabulary_.keys())
topic_words_workaround = {}
for topic, comp in enumerate(lda.components_):
    word_idx = comp.argsort()[-5:][::-1]
    topic_words_workaround[topic] = [feature_names[i] for i in word_idx]

topic_words_workaround

{0: ['total_revolving_bal',
  'contacts_count_12_mon',
  'education_level',
  'avg_open_to_buy',
  'avg_utilization_ratio'],
 1: ['total_trans_amt',
  'naive_bayes_classifier_attrition_flag_card_category_contacts_count_12_mon_dependent_count_education_level_months_inactive_12_mon_2',
  'months_inactive_12_mon',
  'total_trans_ct',
  'attrition_flag'],
 2: ['credit_limit',
  'months_on_book',
  'marital_status',
  'gender',
  'dependent_count'],
 3: ['total_ct_chng_q4_q1',
  'clientnum',
  'naive_bayes_classifier_attrition_flag_card_category_contacts_count_12_mon_dependent_count_education_level_months_inactive_12_mon_1',
  'dependent_count',
  'avg_utilization_ratio'],
 4: ['card_category',
  'income_category',
  'customer_age',
  'total_amt_chng_q4_q1',
  'dependent_count']}

In [16]:
# Assigning names to the topics
topic_names = {
    0: "Demographic Transactions",
    1: "Account & Card Activities",
    2: "Communication & Analysis",
    3: "Customer Engagement Metrics",
    4: "Customer Relationship & Status"
}

# Mapping the topic numbers to their names
named_topics = {topic_names[key]: value for key, value in topic_words_workaround.items()}

named_topics

{'Demographic Transactions': ['total_revolving_bal',
  'contacts_count_12_mon',
  'education_level',
  'avg_open_to_buy',
  'avg_utilization_ratio'],
 'Account & Card Activities': ['total_trans_amt',
  'naive_bayes_classifier_attrition_flag_card_category_contacts_count_12_mon_dependent_count_education_level_months_inactive_12_mon_2',
  'months_inactive_12_mon',
  'total_trans_ct',
  'attrition_flag'],
 'Communication & Analysis': ['credit_limit',
  'months_on_book',
  'marital_status',
  'gender',
  'dependent_count'],
 'Customer Engagement Metrics': ['total_ct_chng_q4_q1',
  'clientnum',
  'naive_bayes_classifier_attrition_flag_card_category_contacts_count_12_mon_dependent_count_education_level_months_inactive_12_mon_1',
  'dependent_count',
  'avg_utilization_ratio'],
 'Customer Relationship & Status': ['card_category',
  'income_category',
  'customer_age',
  'total_amt_chng_q4_q1',
  'dependent_count']}

In [7]:
# Create a DataFrame to display the topics and their top words in a clear manner
topic_df = pd.DataFrame(named_topics).T
topic_df.columns = [f"Word {i+1}" for i in range(topic_df.shape[1])]

# Reset index for better display
topic_df.reset_index(inplace=True)
topic_df.rename(columns={"index": "Topic Name"}, inplace=True)

topic_df

,Topic Name,Word 1,Word 2,Word 3,Word 4,Word 5
0,Demographic Transactions,total_revolving_bal,contacts_count_12_mon,education_level,avg_open_to_buy,avg_utilization_ratio
1,Account & Card Activities,total_trans_amt,naive_bayes_classifier_attrition_flag_card_cat...,months_inactive_12_mon,total_trans_ct,attrition_flag
2,Communication & Analysis,credit_limit,months_on_book,marital_status,gender,dependent_count
3,Customer Engagement Metrics,total_ct_chng_q4_q1,clientnum,naive_bayes_classifier_attrition_flag_card_cat...,dependent_count,avg_utilization_ratio
4,Customer Relationship & Status,card_category,income_category,customer_age,total_amt_chng_q4_q1,dependent_count


In [8]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [17]:
!pip install scikit-learn --upgrade
!pip install pyLDAvis --upgrade

In [9]:
pip install --upgrade scikit-learn pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [18]:
!pip install pyLDAvis==2.1.2

  Using cached pyLDAvis-2.1.2-py2.py3-none-any.whl
  Attempting uninstall: pyLDAvis
    Found existing installation: pyLDAvis 3.4.1
    Uninstalling pyLDAvis-3.4.1:
      Successfully uninstalled pyLDAvis-3.4.1


In [21]:
# Manually add the get_feature_names method to the vectorizer
vectorizer.get_feature_names = vectorizer.get_feature_names_out

# Now, proceed with the pyLDAvis visualization
import pyLDAvis
import pyLDAvis.sklearn

# Display custom topic labels using the topic_names dictionary
for idx, label in topic_names.items():
    print(f"Topic {idx+1}: {label}")

# Prepare the LDA results for visualization
visualization = pyLDAvis.sklearn.prepare(lda, X, vectorizer)

# Display the visualization (this will directly display in a Jupyter notebook)
pyLDAvis.display(visualization)

Topic 1: Demographic Transactions
Topic 2: Account & Card Activities
Topic 3: Communication & Analysis
Topic 4: Customer Engagement Metrics
Topic 5: Customer Relationship & Status
